1. Import libraries 

This is basically my first CNN model that is dedicated to recognizing pneumonia. Let's start with import main libraries.

In [1]:
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
import cv2
import os
%matplotlib inline  

In [1]:
base_dir='../input/chest-xray-pneumonia/'
train_pneumonia=os.listdir('../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/')
train_normal=os.listdir('../input/chest-xray-pneumonia/chest_xray/train/NORMAL/')
val_dir_normal=os.listdir('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/')
test_dir_normal=os.listdir('../input/chest-xray-pneumonia/chest_xray/test/NORMAL/')
val_dir_pneumonia=os.listdir('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/')
test_dir_pneumonia=os.listdir('../input/chest-xray-pneumonia/chest_xray/test/NORMAL/')

Let's see what the lungs look like without and with pneumonia.

2. Data vizualization 

In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img
plt.suptitle('Normal Lungs')
for i in range(0,4):
    img = train_normal[i]
    img = load_img('../input/chest-xray-pneumonia/chest_xray/train/NORMAL/' + img)
    plt.imshow(img)
    plt.show()


In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img
plt.suptitle('Pneumonia')
for i in range(0,4):
    img = train_pneumonia[i]
    img = load_img('../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/' + img)
    plt.imshow(img)
    plt.show()

Viral pneumonia on the X-ray is characterized by such a pattern as Ground-glass opacity. Bacterial pneumonia has clear areas of opacity. In our dataset, we have a lot of people with pneumonia.

![%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.png](attachment:%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.png)

In [1]:
sns.barplot(x=['Normal','Pneumonia'],y=[len(train_normal),len(train_pneumonia)])

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import regularizers

3. Data preprocessing

In [1]:
train_datagen = ImageDataGenerator(    rescale=1/255.,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        rotation_range=0.4,
                                        zoom_range=0.4
                                                                       )

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train/',
                                                 target_size=(224, 224),
                                                 batch_size=8,
                                                 class_mode='binary')

val_set = train_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/val',
                                            target_size=(224, 224),
                                            batch_size=8,
                                            shuffle=True,
                                            class_mode='binary')

test_set = test_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',
                                            target_size=(224, 224),
                                            batch_size=8,
                                            class_mode='binary')


In [1]:
# Learning rate optimization using tensorflow callbacks
# Our Learning rate varies from 0.01 to 1 and we are increasing or decreasing
# this learning rate with factor of 0.1 to adjust.

# types of callbacks are available in tensorflow framework

In [1]:
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau

RLR = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.1,
                                         patience=3,
                                         cooldown=3,
                                         min_lr=0.01,
                                        verbose=1)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98):
            print("\nReached 98% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = [RLR,myCallback()]

In [1]:
# We are taking input shape of tensor as 224*224*3

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras import optimizers
from keras import backend as K
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)))
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
model.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4),
    activity_regularizer=regularizers.l2(1e-5)))
print(model.summary())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(training_set,
                    steps_per_epoch=5216//8,
                    epochs=30,
                    validation_data=test_set,
                    validation_steps=624//8,
                    callbacks=callbacks)

In [1]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss'
        )
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()


In [1]:
ev = model.evaluate_generator(val_set)
print("\n%s: %.f%%" % (model.metrics_names[1], ev[1]*100))

In [1]:
# y_pred = model.predict(test_set)
# ev

In [1]:
# from sklearn.metrics import f1_score
# ac= f1_score(y_test, y_pred.round())
# print('accuracy of the model: ',ac)

In [1]:
# y_pred.round()

In [1]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [1]:
model.save('new_model.h5')